# Module Dependency Resolution and Modprobe

Let's reuse the [Exporting & Invoking Linux Kernel Symbols](export_kernel_symbols.ipynb) to make two kernel modules that one depend on another: the `hello_feature.ko` will call a function `api_test()` that's define within module `hello_api.ko`.

In [1]:
! make

make -C /lib/modules/6.1.29/build M=/home/clymber/Studio/kerdevel/export_symbol modules
make[1]: Entering directory '/home/clymber/Studio/linux-6.1.29'
make[1]: Leaving directory '/home/clymber/Studio/linux-6.1.29'


## 1. The command `insmod` and `rmmod`

Since `hello_feature.ko` depends on module `hello_api.ko`, so if we install `hello_api.ko` before we install `hello_feature.ko`, everything will go on very well:

In [2]:
! sudo dmesg -C
! sudo insmod hello_api.ko && sudo insmod hello_feature.ko
! sudo rmmod hello_feature && sudo rmmod hello_api
! sudo dmesg -c

[ 4715.437537] [hello_api] Hello, World!
[ 4715.443888] [hello_feature] Hello, World!
[ 4715.443890] [hello_api] this is a test message.
[ 4715.553594] [hello_feature] Bye Bye, World!
[ 4715.598908] [hello_api] Bye bye, World!


But if module `hello_feature.ko` depends on lots of other modules, and even worse if the other modules depend on each other, things will be getting complicated. What if we just install the module `hello_feature.ko` and left the other alone?

In [3]:
! sudo insmod hello_feature.ko; sudo dmesg -c

insmod: ERROR: could not insert module hello_feature.ko: Unknown symbol in module
[ 4715.854285] hello_feature: Unknown symbol api_test (err -2)


The command `insmod` simply asked the kernel to load the module we specify. The `insmod` program does not perform any dependency resolution or advanced error checking.

In [4]:
! modinfo --field depends hello_feature.ko

hello_api


The command output above shows that *hello_feature.ko* depends on another module named *hello_api*.

## 2. The command `modprobe`

The utility `modprobe` provides dependency resolution, error checking and reporting, configurable behavior, and more advanced features. Its general forms of usage:
```sh
sudo modprobe <options> <module name>
# or
sudo modprobe <options> -a <first module name> <second module name>
```
For more information, see [The Modprobe Command In Linux](https://phoenixnap.com/kb/modprobe-command).

So, let's have a try:

In [5]:
! sudo modprobe hello_feature.ko; sudo dmesg -c

modprobe: FATAL: Module hello_feature.ko not found in directory /lib/modules/6.1.29


Oops! It didn't work as expected. The `/lib/modules/<kernel release>` indicates a directory where the system kernel modules reside.

In [6]:
! strace modprobe hello_feature.ko 2>&1 | grep "/lib/modules/$(uname -r)"

openat(AT_FDCWD, "/lib/modules/6.1.29/modules.softdep", O_RDONLY|O_CLOEXEC) = 3
openat(AT_FDCWD, "/lib/modules/6.1.29/modules.dep.bin", O_RDONLY|O_CLOEXEC) = 3
openat(AT_FDCWD, "/lib/modules/6.1.29/modules.alias.bin", O_RDONLY|O_CLOEXEC) = 3
openat(AT_FDCWD, "/lib/modules/6.1.29/modules.symbols.bin", O_RDONLY|O_CLOEXEC) = 3
openat(AT_FDCWD, "/lib/modules/6.1.29/modules.builtin.alias.bin", O_RDONLY|O_CLOEXEC) = 3
openat(AT_FDCWD, "/lib/modules/6.1.29/modules.builtin.bin", O_RDONLY|O_CLOEXEC) = 3
write(2, "modprobe: FATAL: Module hello_fe"..., 84modprobe: FATAL: Module hello_feature.ko not found in directory /lib/modules/6.1.29


The `strace` utility shows that it highly depends on the module installing directory of current version kernel.  

## 3. Generating Dependency

Why was that? The answer was given in the book _Linux Kernel Development_:  
> "The dependency information must be generated. Linux distributions generate the mapping automatically and keep it up to date on each boot.To build the module dependency information, as root simply run `depmod`.  
> To perform a quick update, rebuilding only the information for modules newer than the dependency information, run as root `depmod -A`. The module dependency information is stored in the file `/lib/modules/version/modules.dep`."

What we need to do is to install the modules into `/lib/modules/<kernel release>` and to update the dependency information.

In [7]:
! sudo cp {hello_api,hello_feature}.ko /lib/modules/$(uname -r)/
! sudo sudo depmod -a
! sudo modprobe hello_feature; sudo dmesg -c

[ 4717.378259] [hello_api] Hello, World!
[ 4717.378438] [hello_feature] Hello, World!


In [8]:
! sudo modprobe -r hello_feature; sudo dmesg -c

[ 4717.378438] [hello_api] this is a test message.
[ 4717.501102] [hello_feature] Bye Bye, World!
[ 4717.533367] [hello_api] Bye bye, World!


It worked! Actually, we can use the build system of Linux kernel source tree to simplify the toiling work:

In [9]:
! sudo make modules_install

make -C /lib/modules/6.1.29/build M=/home/clymber/Studio/kerdevel/export_symbol modules_install
make[1]: Entering directory '/home/clymber/Studio/linux-6.1.29'
  INSTALL /lib/modules/6.1.29/extra/hello_api.ko
  SIGN    /lib/modules/6.1.29/extra/hello_api.ko
  INSTALL /lib/modules/6.1.29/extra/hello_feature.ko
  SIGN    /lib/modules/6.1.29/extra/hello_feature.ko
  DEPMOD  /lib/modules/6.1.29
make[1]: Leaving directory '/home/clymber/Studio/linux-6.1.29'


In [10]:
! sudo modprobe hello_feature; sudo dmesg -c
! sudo modprobe -r hello_feature; sudo dmesg -c

[ 4718.698889] [hello_api] Hello, World!
[ 4718.699639] [hello_feature] Hello, World!
[ 4718.699641] [hello_api] this is a test message.
[ 4718.818173] [hello_feature] Bye Bye, World!
[ 4718.845432] [hello_api] Bye bye, World!


## 4. Cleaning up

In [11]:
! sudo rm /lib/modules/$(uname -r)/extra/hello_{feature,api}.ko
! sudo depmod -a
! make clean

make -C /lib/modules/6.1.29/build M=/home/clymber/Studio/kerdevel/export_symbol clean
make[1]: Entering directory '/home/clymber/Studio/linux-6.1.29'
  CLEAN   /home/clymber/Studio/kerdevel/export_symbol/Module.symvers
make[1]: Leaving directory '/home/clymber/Studio/linux-6.1.29'
